<a href="https://colab.research.google.com/github/zharfanw/fd_zharram/blob/main/FDProject_MinimizePower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fullduplex Project : Transmit Power Minimization for Double-RIS-Enabled ISAC System**

This notebook referenced by this paper [Transmit Power Minimization for Double-RIS-Enabled ISAC System](https://ieeexplore.ieee.org/document/10681845)


In [ ]:
!pip install cvxpy
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user
# !jupyter nbextension enable varInspector/main

# Symbol List


In [ ]:
import numpy as np
import cvxpy as cp

## Function : Initial Parameters


Initialize the values of $ \{ \mathbf{w}_{c,k}, \mathbf{w}_{r,m}, \mathbf{v}_1, \mathbf{v}_2 \}$ to $
 \{ \mathbf{w}_{c,k}^0, \mathbf{w}_{r,m}^0, \mathbf{v}_1^0, \mathbf{v}_2^0 \}, $ respectively

 where

 $\mathbf{v}_1 \in \mathbb{C}^{N_1 \times 1}$ is the phase shift vector for RIS1, defined as:

$$
\mathbf{v}_1 = [\psi_1, \ldots, \psi_{N_1}]^H
$$

With the constraint:

$$
|\mathbf{v}_1[j]| = 1, \quad \forall j = 1, \ldots, N_1
$$

This means that each element of $\mathbf{v}_1$ has unit modulus (magnitude 1), i.e., it only adjusts the phase of the reflected signal without changing its amplitude.

Similarly, $\mathbf{v}_2$ is the phase shift vector for RIS2 (if applicable), also with:

$$
|\mathbf{v}_2[j]| = 1, \quad \forall j
$$



In [ ]:
def initialize_parameters(K, M, N1, N2):
  # K, num of user,
  # M, num of transmit antenna
  # N1 and N2, num of elements in RIS1 and RIS2
  # bener
    """ Initialize beamforming vectors and phase shifts """
    wc = [np.random.randn(M, 1) + 1j * np.random.randn(M, 1) for _ in range(K)] # K jumlah user
    wr = [np.random.randn(M, 1) + 1j * np.random.randn(M, 1) for _ in range(M)]
    v1 = np.exp(1j * 2 * np.pi * np.random.rand(N1, 1))
    v2 = np.exp(1j * 2 * np.pi * np.random.rand(N2, 1))
    return wc, wr, v1, v2

##  Function : Total Transmit Power

Calculate the total transmit power $P_b^\nu$:

$$
P_b^\nu \triangleq \sum_{k \in \mathcal{K}} \|\mathbf{w}_{c,k}\|^2 + \sum_{m \in \mathcal{M}} \|\mathbf{w}_{r,m}\|^2
$$
based on equation (2):
$$
\mathbb{E}\{\mathbf{x}^H \mathbf{x}\} = \sum_{k \in \mathcal{K}} \|\mathbf{w}_{c,k}\|^2 + \sum_{m \in \mathcal{M}} \|\mathbf{w}_{r,m}\|^2.
$$

In [ ]:
def total_transmit_power(wc, wr):
    """ Calculate the total transmit power """
    # l2norm wc
    l2norm_wc = [np.linalg.norm(w) for w in wc]
    l2norm_wr = [np.linalg.norm(w) for w in wr]
    Pb_nu = sum(l2norm_wc) + sum(l2norm_wr)
    return Pb_nu

## Function : Update Beamformer $\{\mathbf{w}_{c,k}\}^{\nu+1}$ and $\{\mathbf{w}_{r,m}\}^{\nu+1}$ with given $\mathbf{v}_1^\nu$ and $\mathbf{v}_2^\nu$

\begin{aligned}
\min_{\{\mathbf{w}_{c,k}\}, \{\mathbf{w}_{r,m}\}} \quad & \sum_{k \in \mathcal{K}} \|\mathbf{w}_{c,k}\|^2 + \sum_{m \in \mathcal{M}} \|\mathbf{w}_{r,m}\|^2 \quad \text{(17)}\\
\text{s.t.}
\quad & \Upsilon_R \sigma_R^2 \leq \sum_{k \in \mathcal{K}} F_A(\mathbf{w}_{c,k}, \widetilde{\mathbf{w}}_{c,k}) + \sum_{m \in \mathcal{M}} F_A(\mathbf{w}_{r,m}, \widetilde{\mathbf{w}}_{r,m}) \quad \text{(15)} \\
&
(2^{\Gamma_k} - 1) \left( \sum_{m \in \mathcal{M}} \mathbf{w}_{r,m}^H \mathbf{B} \mathbf{w}_{r,m} + \sum_{i \neq k, i \in \mathcal{K}} \mathbf{w}_{c,i}^H \mathbf{B} \mathbf{w}_{c,i} + \sigma_k^2 \right) \leq F_B(\mathbf{w}_{c,k}, \widetilde{\mathbf{w}}_{c,k}) \quad \text{(16)}
\end{aligned}

Where equation (15) and (16) are result of first approximation of equation below

$
\Upsilon_R \sigma_R^2 \leq \sum_{k \in \mathcal{K}} \mathbf{w}_{c,k}^H \mathbf{A} \mathbf{w}_{c,k} + \sum_{m \in \mathcal{M}} \mathbf{w}_{r,m}^H \mathbf{A} \mathbf{w}_{r,m} \tag{12}
$

$
(2^{\Gamma_k} - 1) \left( \sum_{m \in \mathcal{M}} \mathbf{w}_{r,m}^H \mathbf{B} \mathbf{w}_{r,m} + \sum_{i \neq k, i \in \mathcal{K}} \mathbf{w}_{c,i}^H \mathbf{B} \mathbf{w}_{c,i} + \sigma_k^2 \right) \leq \mathbf{w}_{c,k}^H \mathbf{B} \mathbf{w}_{c,k} \tag{13}
$

where $
 \quad \mathbf{A} \triangleq |\alpha|^2 \mathbf{H}_R^H \mathbf{H}_R \succeq 0, \quad \mathbf{B} \triangleq \mathbf{h}_k \mathbf{h}_k^H \succeq 0.
$

$
\mathbf{H}_R = \left( \mathbf{a}(\theta_b) + \mathbf{H}_1^H \operatorname{diag}(\mathbf{v}_1) \mathbf{a}(\theta_r) \right) \left( \mathbf{a}^H(\theta_b) + \mathbf{a}^H(\theta_r) \operatorname{diag}(\mathbf{v}_1) \mathbf{H}_1 \right)
\tag{based on 3}
$

$
\mathbf{H}_1 = \sqrt{\frac{\epsilon_1}{1 + \epsilon_1}} \mathbf{H}_1^{\mathrm{LOS}} + \sqrt{\frac{1}{1 + \epsilon_1}} \mathbf{H}_1^{\mathrm{NLOS}} \tag{4}
$

$H_1^{LOS}$ is Channel MIMO Line-of-Sight. Referenced by Tse and Viswanath: Fundamentals of Wireless Communication pages 352 Eq. 7.29,

$$
\mathbf{H^{LOS}} = a \cdot \sqrt{n_t n_r} \cdot \exp\left( -j \frac{2\pi d}{\lambda_c} \right) \cdot \mathbf{e}_r(\Omega_r) \cdot \mathbf{e}_t^*(\Omega_t)
$$
where $a = \sqrt{P_t \cdot G_t \cdot G_r} \cdot \left( \frac{\lambda}{4\pi d} \right)$ as Friis Free Space Transmission Equation. $n_t n_r$ is the power gain of the MIMO Channel, or simply $n_t$ is num of transmit antenna, $n_r$ is num of receive antenna. $\lambda_c$ is length of cerrier wave in meter. And $d$ is distance between BS and RIS1

and $H_1^{NLOS}$ represent Reyleight fading non-LOS Component, which has i.i.d circularly symmetruc complex Gaussioan entries with zero-mean and unit variance

In [ ]:
def array_response_ULA(N, theta, d=0.5):
    """
    Menghitung array response untuk Uniform Linear Array (ULA)

    N     : Jumlah antena
    theta : Sudut datang (radian)
    d     : Spasi antar elemen (dalam satuan lambda, default lambda/2)
    """
    n = np.arange(N)
    return np.exp(1j * 2 * np.pi * d * n * np.sin(theta))

def H1_LOS_Exact(N_r, N_t, theta_r, theta_t, d_prop=10, d_ant=0.5, fc=3e9, PL0_dB=-30, tau=2.2, d0=1):
    """
    Menghitung kanal H1^LOS sesuai persamaan:
    H = |a| * exp(-j*2*pi*d/lambda_c) * sqrt(N_t * N_r) * e_r(Ω_r) * e_t^H(Ω_t)

    N_r     : Jumlah elemen RIS1 (receiver)
    N_t     : Jumlah antena BS (transmitter)
    theta_r : Sudut AoA di RIS1 (radian)
    theta_t : Sudut AoD dari BS (radian)
    d_prop  : Jarak propagasi (meter)
    d_ant   : Spasi antar elemen array (default 0.5 lambda)
    fc      : Frekuensi carrier (Hz)
    PL0_dB  : Path loss pada d0 (default -30 dB)
    tau     : Path loss exponent
    d0      : Referensi jarak (default 1 meter)
    """
    c = 3e8  # kecepatan cahaya (m/s)
    lambda_c = c / fc

    # Steering vectors
    e_r = array_response_ULA(N_r, theta_r, d_ant)
    e_t = array_response_ULA(N_t, theta_t, d_ant)

    # Path loss in dB
    PL_dB = PL0_dB - 10 * tau * np.log10(d_prop / d0)

    # Gain a = |a| * exp(-j 2pi d / lambda)
    a_abs = 10**(PL_dB / 20)  # amplitudo
    a_phase = np.exp(-1j * 2 * np.pi * d_prop / lambda_c)
    a = a_abs * a_phase

    # H1^LOS
    H_los = a * np.sqrt(N_t * N_r) * np.outer(e_r, np.conj(e_t))

    return H_los

# # Contoh penggunaan:
# N_r = 30  # RIS1
# N_t = 8   # BS
# fc = 28e9  # 28 GHz
# d_prop = 2  # meter, sesuai skenario paper
# tau = 2.2

# theta_r = np.deg2rad(45)  # AoA di RIS1
# theta_t = np.deg2rad(30)  # AoD dari BS

# H1_los = H1_LOS_Exact(N_r, N_t, theta_r, theta_t, d_prop=d_prop, fc=fc, tau=tau)
# print("H1^LOS shape:", H1_los.shape)


In [ ]:
def update_beamformers(wc, wr, v1, v2):
    """ Placeholder for beamforming update step (solve problem 17) """
    # This needs to be implemented based on specific optimization requirements
    return wc, wr

In [ ]:


def transform_subproblem(wc, wr, v1, v2):
    """ Transform subproblem (18) into (32) """
    # Placeholder for transformation step
    U1 = np.random.rand(len(v1), len(v1))
    return U1

def solve_sdr(U1):
    """ Solve SDR for given matrix U1 """
    n = U1.shape[0]
    u1 = cp.Variable((n, n), complex=True)
    constraints = [u1 >> 0, cp.trace(u1) == 1]
    prob = cp.Problem(cp.Minimize(cp.real(cp.trace(U1 @ u1))), constraints)
    prob.solve()
    return u1.value

def gaussian_randomization(U1):
    """ Obtain phase shifts v1 from U1 using Gaussian randomization """
    eig_vals, eig_vecs = np.linalg.eig(U1)
    max_eig_vec = eig_vecs[:, np.argmax(eig_vals)]
    v1 = np.exp(1j * np.angle(max_eig_vec))
    return v1

K = 3  # Number of users
M = 8  # Number of antennas at BS
N1, N2 = 10, 10  # Number of elements in RIS1 and RIS2
epsilon = 1e-2  # Convergence threshold 0.01


wc, wr, v1, v2 = initialize_parameters(K, M, N1, N2)
Pb_nu = total_transmit_power(wc, wr)
nu = 0 # nu in greek



# while True:
#     wc, wr = update_beamformers(wc, wr, v1, v2)
#     U1 = transform_subproblem(wc, wr, v1, v2)
#     U1_solved = solve_sdr(U1)
#     # v1 = gaussian_randomization(U1_solved)
#     # Pb_nu_plus1 = total_transmit_power(wc, wr)

#     # if abs(Pb_nu_plus1 - Pb_nu) < epsilon:
#     #     break

#     # Pb_nu = Pb_nu_plus1
#     # nu += 1
#     # print(f"Nu {nu}: Transmit Power = {Pb_nu_plus1}")